In [ ]:
import pennylane as qml
from pennylane import numpy as np
from typing import List, Tuple
import networkx as nx

class VRPQAOA:
    def __init__(self, n_locations: int, n_vehicles: int, distance_matrix: np.ndarray):
        """
        Initialize VRP QAOA solver
        
        Args:
            n_locations: Number of locations (including depot)
            n_vehicles: Number of vehicles
            distance_matrix: Matrix of distances between locations
        """
        self.n_locations = n_locations
        self.n_vehicles = n_vehicles
        self.distance_matrix = distance_matrix
        
        # Number of qubits needed = n_locations * (n_locations-1)
        self.n_qubits = n_locations * (n_locations - 1)
        
        # Initialize the device
        self.dev = qml.device("lightning.qubit", wires=self.n_qubits)
        
    def _get_ising_hamiltonian(self) -> Tuple[np.ndarray, np.ndarray, float]:
        """
        Convert VRP to Ising formulation
        
        Returns:
            J: Coupling matrix
            h: Local field vector
            c: Constant offset
        """
        # Initialize matrices
        Q = np.zeros((self.n_qubits, self.n_qubits))
        g = np.zeros(self.n_qubits)
        
        # Penalty coefficient for constraints
        A = np.max(self.distance_matrix) * 10
        
        # Create helper matrices for constraints
        for i in range(self.n_locations):
            # Source constraints
            source_indices = [j for j in range(self.n_locations) if j != i]
            for idx1 in source_indices:
                for idx2 in source_indices:
                    if idx1 != idx2:
                        Q[i*(self.n_locations-1) + idx1][i*(self.n_locations-1) + idx2] += A
                
            # Target constraints
            target_indices = [j for j in range(self.n_locations) if j != i]
            for idx1 in target_indices:
                for idx2 in target_indices:
                    if idx1 != idx2:
                        Q[idx1*(self.n_locations-1) + i][idx2*(self.n_locations-1) + i] += A
                        
        # Add distance terms
        for i in range(self.n_locations):
            for j in range(self.n_locations):
                if i != j:
                    idx = i*(self.n_locations-1) + (j if j < i else j-1)
                    g[idx] += self.distance_matrix[i][j]
        
        # Vehicle constraints for depot
        depot_source_indices = range(self.n_locations-1)
        depot_target_indices = range(self.n_locations-1)
        
        for idx1 in depot_source_indices:
            for idx2 in depot_source_indices:
                if idx1 != idx2:
                    Q[idx1][idx2] += A
                    
        for idx1 in depot_target_indices:
            for idx2 in depot_target_indices:
                if idx1 != idx2:
                    Q[idx1*(self.n_locations-1)][idx2*(self.n_locations-1)] += A
        
        # Convert QUBO to Ising
        J = -Q/4
        h = g/2 + np.sum(Q/4, axis=1)
        c = np.sum(g/2) + np.sum(Q/4)
        
        return J, h, c
        
    def cost_hamiltonian(self, params, wires):
        """
        Implement the cost Hamiltonian
        """
        J, h, _ = self._get_ising_hamiltonian()
        
        # Add ZZ interactions
        for i in range(self.n_qubits):
            for j in range(i+1, self.n_qubits):
                if abs(J[i,j]) > 1e-10:
                    qml.CNOT(wires=[i, j])
                    qml.RZ(2 * J[i,j] * params, wires=j)
                    qml.CNOT(wires=[i, j])
        
        # Add Z terms
        for i in range(self.n_qubits):
            if abs(h[i]) > 1e-10:
                qml.RZ(2 * h[i] * params, wires=i)

    def mixer_hamiltonian(self, params, wires):
        """
        Implement the mixer Hamiltonian
        """
        for wire in wires:
            qml.RX(2 * params, wires=wire)

    def circuit(self, params, steps):
        """
        Implement QAOA circuit
        
        Args:
            params: List of gamma and beta parameters
            steps: Number of QAOA steps (p)
            
        Returns:
            Expectation value of the cost Hamiltonian
        """
        @qml.qnode(self.dev)
        def qaoa_circuit(params, steps):
            # Initialize in superposition
            for wire in range(self.n_qubits):
                qml.Hadamard(wires=wire)
                
            # Implement QAOA steps
            for i in range(steps):
                self.cost_hamiltonian(params[i], range(self.n_qubits))
                self.mixer_hamiltonian(params[steps + i], range(self.n_qubits))
                
            return qml.expval(qml.PauliZ(0))  # Measure first qubit
        
        return qaoa_circuit(params, steps)
    
    def optimize(self, steps: int, n_iterations: int = 100) -> Tuple[np.ndarray, float]:
        """
        Optimize the QAOA parameters
        
        Args:
            steps: Number of QAOA steps (p)
            n_iterations: Number of optimization iterations
            
        Returns:
            optimal_params: Optimal gamma and beta parameters
            min_cost: Minimum cost found
        """
        # Initialize parameters
        init_params = np.random.uniform(0, 2*np.pi, 2*steps)
        
        # Define cost function
        def cost(params):
            return self.circuit(params, steps)
        
        # Optimize parameters
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        params = init_params
        
        for i in range(n_iterations):
            params = opt.step(cost, params)
            
        return params, cost(params)

    def get_solution(self, optimal_params: np.ndarray, steps: int) -> List[List[int]]:
        """
        Get the VRP solution from optimal parameters
        
        Args:
            optimal_params: Optimal QAOA parameters
            steps: Number of QAOA steps used
            
        Returns:
            routes: List of vehicle routes
        """
        # Run circuit with optimal parameters and measure all qubits
        @qml.qnode(self.dev)
        def measure_circuit(params, steps):
            # Initialize in superposition
            for wire in range(self.n_qubits):
                qml.Hadamard(wires=wire)
                
            # Implement QAOA steps
            for i in range(steps):
                self.cost_hamiltonian(params[i], range(self.n_qubits))
                self.mixer_hamiltonian(params[steps + i], range(self.n_qubits))
                
            return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
        
        measurements = measure_circuit(optimal_params, steps)
        
        # Convert measurements to binary
        binary_solution = [1 if m > 0 else 0 for m in measurements]
        
        # Convert binary solution to routes
        routes = []
        current_route = [0]  # Start at depot
        
        # Extract routes from binary solution
        for i in range(self.n_vehicles):
            while len(current_route) < self.n_locations:
                current = current_route[-1]
                next_idx = -1
                
                # Find next location in route
                for j in range(self.n_locations):
                    if j not in current_route:
                        idx = current*(self.n_locations-1) + (j if j < current else j-1)
                        if binary_solution[idx] == 1:
                            next_idx = j
                            break
                            
                if next_idx == -1:
                    break
                    
                current_route.append(next_idx)
                
            if len(current_route) > 1:
                routes.append(current_route)
                current_route = [0]  # Start new route at depot
                
        return routes

    def get_state_probabilities(self, optimal_params: np.ndarray, steps: int, n_shots: int = 1000) -> dict:
        """
        Get probability distribution of measured states
        
        Args:
            optimal_params: Optimal QAOA parameters
            steps: Number of QAOA steps used
            n_shots: Number of circuit measurements
            
        Returns:
            probabilities: Dictionary mapping state indices to their probabilities
        """
        @qml.qnode(self.dev)
        def measurement_circuit(params, steps):
            # Initialize in superposition
            for wire in range(self.n_qubits):
                qml.Hadamard(wires=wire)
                
            # Implement QAOA steps
            for i in range(steps):
                self.cost_hamiltonian(params[i], range(self.n_qubits))
                self.mixer_hamiltonian(params[steps + i], range(self.n_qubits))
                
            return qml.sample()
        
        # Take multiple shots and get state probabilities
        results = []
        for _ in range(n_shots):
            measurements = measurement_circuit(optimal_params, steps)
            # Convert binary measurements to decimal index
            state_idx = sum([m * 2**i for i, m in enumerate(measurements)])
            results.append(state_idx)
        
        # Calculate probabilities
        unique_states, counts = np.unique(results, return_counts=True)
        probabilities = {int(state): count/n_shots for state, count in zip(unique_states, counts)}
        
        return probabilities

    def find_optimal_states(self, probabilities: dict) -> List[int]:
        """
        Find indices of optimal states from probability distribution
        
        Args:
            probabilities: Dictionary of state probabilities
            
        Returns:
            optimal_states: List of optimal state indices
        """
        # Calculate cost for each state
        costs = {}
        for state_idx in probabilities.keys():
            # Convert decimal index to binary state
            binary_state = [int(x) for x in format(state_idx, f'0{self.n_qubits}b')]
            
            # Check if state is feasible (satisfies constraints)
            if self._is_feasible_state(binary_state):
                cost = self._calculate_cost(binary_state)
                costs[state_idx] = cost
        
        # Find states with minimum cost
        if not costs:
            return []
        
        min_cost = min(costs.values())
        optimal_states = [state for state, cost in costs.items() if abs(cost - min_cost) < 1e-6]
        
        return optimal_states

    def _is_feasible_state(self, binary_state: List[int]) -> bool:
        """
        Check if state satisfies VRP constraints
        """
        # Check each location is visited exactly once
        for i in range(1, self.n_locations):
            # Check outgoing edges
            outgoing = sum(binary_state[i*(self.n_locations-1):(i+1)*(self.n_locations-1)])
            if outgoing != 1:
                return False
            
            # Check incoming edges
            incoming = sum(binary_state[j*(self.n_locations-1) + (i if i < j else i-1)] 
                        for j in range(self.n_locations) if j != i)
            if incoming != 1:
                return False
        
        # Check vehicle count constraint (edges from/to depot)
        depot_outgoing = sum(binary_state[:self.n_locations-1])
        depot_incoming = sum(binary_state[i*(self.n_locations-1)] 
                            for i in range(1, self.n_locations))
        
        return depot_outgoing == self.n_vehicles and depot_incoming == self.n_vehicles

    def _calculate_cost(self, binary_state: List[int]) -> float:
        """
        Calculate cost of a given state
        """
        cost = 0
        for i in range(self.n_locations):
            for j in range(self.n_locations):
                if i != j:
                    idx = i*(self.n_locations-1) + (j if j < i else j-1)
                    if binary_state[idx] == 1:
                        cost += self.distance_matrix[i][j]
        return cost

    def visualize_distribution(self, probabilities: dict, optimal_states: List[int]):
        """
        Visualize probability distribution and highlight optimal states
        """
        import matplotlib.pyplot as plt
        
        # Create bar plot
        plt.figure(figsize=(12, 6))
        states = list(probabilities.keys())
        probs = list(probabilities.values())
        
        # Plot all states
        plt.bar(states, probs, color='gray', alpha=0.5)
        
        # Highlight optimal states
        optimal_probs = [probabilities[state] for state in optimal_states]
        plt.bar(optimal_states, optimal_probs, color='blue', 
                label='Optimal States')
        
        plt.xlabel('State Index')
        plt.ylabel('Probability')
        plt.title(f'Probability Distribution for VRP({self.n_locations},{self.n_vehicles})')
        plt.legend()
        
        # Add cost labels for optimal states
        for state in optimal_states:
            binary_state = [int(x) for x in format(state, f'0{self.n_qubits}b')]
            cost = self._calculate_cost(binary_state)
            plt.text(state, probabilities[state], f'Cost: {cost:.3f}', 
                    ha='center', va='bottom')
        
        plt.show()

# Example usage:
if __name__ == "__main__":
    # Example problem instance (4,2) from the paper
    n_locations = 4
    n_vehicles = 2
    distances = np.array([
        [0.0, 36.84, 5.06, 30.63],
        [36.84, 0.0, 24.55, 63.22],
        [5.06, 24.55, 0.0, 15.50],
        [30.63, 63.22, 15.50, 0.0]
    ])
    
    # Create and run VRP QAOA solver
    vrp_solver = VRPQAOA(n_locations, n_vehicles, distances)
    optimal_params, min_cost = vrp_solver.optimize(steps=12)
    
    # Get and visualize probability distribution
    probabilities = vrp_solver.get_state_probabilities(optimal_params, steps=12)
    optimal_states = vrp_solver.find_optimal_states(probabilities)
    
    print(f"Optimal state indices: {optimal_states}")
    vrp_solver.visualize_distribution(probabilities, optimal_states)
    
    # Print routes for optimal states
    for state_idx in optimal_states:
        binary_state = [int(x) for x in format(state_idx, f'0{vrp_solver.n_qubits}b')]
        routes = vrp_solver._binary_to_routes(binary_state)
        cost = vrp_solver._calculate_cost(binary_state)
        print(f"\nState {state_idx}:")
        print(f"Routes: {routes}")
        print(f"Cost: {cost}")
    print(f"Minimum cost: {min_cost}")

/Users/yousefelbrolosy/anaconda3/envs/qiskit-stable8/lib/python3.11/site-packages/pennylane/_grad.py:157: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


TypeError: generate_samples(): incompatible function arguments. The following argument types are supported:
    1. (self: pennylane_lightning.lightning_qubit_ops.MeasurementsC128, arg0: int, arg1: int) -> numpy.ndarray

Invoked with: <pennylane_lightning.lightning_qubit_ops.MeasurementsC128 object at 0x314cf54b0>, 12, None

In [5]:
import pennylane as qml
from pennylane import numpy as np
from typing import List, Tuple
import networkx as nx

class VRPQAOA:
    def __init__(self, n_locations: int, n_vehicles: int, distance_matrix: np.ndarray):
        """
        Initialize VRP QAOA solver
        
        Args:
            n_locations: Number of locations (including depot)
            n_vehicles: Number of vehicles
            distance_matrix: Matrix of distances between locations
        """
        self.n_locations = n_locations
        self.n_vehicles = n_vehicles
        self.distance_matrix = distance_matrix
        
        # Number of qubits needed = n_locations * (n_locations-1)
        self.n_qubits = n_locations * (n_locations - 1)
        
        # Initialize the device
        self.dev = qml.device("lightning.qubit", wires=self.n_qubits)
        
    def _get_ising_hamiltonian(self) -> Tuple[np.ndarray, np.ndarray, float]:
        """
        Convert VRP to Ising formulation
        
        Returns:
            J: Coupling matrix
            h: Local field vector
            c: Constant offset
        """
        # Initialize matrices
        Q = np.zeros((self.n_qubits, self.n_qubits))
        g = np.zeros(self.n_qubits)
        
        # Penalty coefficient for constraints
        A = np.max(self.distance_matrix) * 10
        
        # Create helper matrices for constraints
        for i in range(self.n_locations):
            # Source constraints
            source_indices = [j for j in range(self.n_locations) if j != i]
            for idx1 in source_indices:
                for idx2 in source_indices:
                    if idx1 != idx2:
                        Q[i*(self.n_locations-1) + idx1][i*(self.n_locations-1) + idx2] += A
                
            # Target constraints
            target_indices = [j for j in range(self.n_locations) if j != i]
            for idx1 in target_indices:
                for idx2 in target_indices:
                    if idx1 != idx2:
                        Q[idx1*(self.n_locations-1) + i][idx2*(self.n_locations-1) + i] += A
                        
        # Add distance terms
        for i in range(self.n_locations):
            for j in range(self.n_locations):
                if i != j:
                    idx = i*(self.n_locations-1) + (j if j < i else j-1)
                    g[idx] += self.distance_matrix[i][j]
        
        # Vehicle constraints for depot
        depot_source_indices = range(self.n_locations-1)
        depot_target_indices = range(self.n_locations-1)
        
        for idx1 in depot_source_indices:
            for idx2 in depot_source_indices:
                if idx1 != idx2:
                    Q[idx1][idx2] += A
                    
        for idx1 in depot_target_indices:
            for idx2 in depot_target_indices:
                if idx1 != idx2:
                    Q[idx1*(self.n_locations-1)][idx2*(self.n_locations-1)] += A
        
        # Convert QUBO to Ising
        J = -Q/4
        h = g/2 + np.sum(Q/4, axis=1)
        c = np.sum(g/2) + np.sum(Q/4)
        
        return J, h, c
        
    def cost_hamiltonian(self, params, wires):
        """
        Implement the cost Hamiltonian
        """
        J, h, _ = self._get_ising_hamiltonian()
        
        # Add ZZ interactions
        for i in range(self.n_qubits):
            for j in range(i+1, self.n_qubits):
                if abs(J[i,j]) > 1e-10:
                    qml.CNOT(wires=[i, j])
                    qml.RZ(2 * J[i,j] * params, wires=j)
                    qml.CNOT(wires=[i, j])
        
        # Add Z terms
        for i in range(self.n_qubits):
            if abs(h[i]) > 1e-10:
                qml.RZ(2 * h[i] * params, wires=i)

    def mixer_hamiltonian(self, params, wires):
        """
        Implement the mixer Hamiltonian
        """
        for wire in wires:
            qml.RX(2 * params, wires=wire)

    def circuit(self, params, steps):
        """
        Implement QAOA circuit
        
        Args:
            params: List of gamma and beta parameters
            steps: Number of QAOA steps (p)
            
        Returns:
            Expectation value of the cost Hamiltonian
        """
        @qml.qnode(self.dev)
        def qaoa_circuit(params, steps):
            # Initialize in superposition
            for wire in range(self.n_qubits):
                qml.Hadamard(wires=wire)
                
            # Implement QAOA steps
            for i in range(steps):
                self.cost_hamiltonian(params[i], range(self.n_qubits))
                self.mixer_hamiltonian(params[steps + i], range(self.n_qubits))
                
            return qml.expval(qml.PauliZ(0))  # Measure first qubit
        
        return qaoa_circuit(params, steps)
    
    def optimize(self, steps: int, n_iterations: int = 100) -> Tuple[np.ndarray, float]:
        """
        Optimize the QAOA parameters
        
        Args:
            steps: Number of QAOA steps (p)
            n_iterations: Number of optimization iterations
            
        Returns:
            optimal_params: Optimal gamma and beta parameters
            min_cost: Minimum cost found
        """
        # Initialize parameters
        init_params = np.random.uniform(0, 2*np.pi, 2*steps)
        
        # Define cost function
        def cost(params):
            return self.circuit(params, steps)
        
        # Optimize parameters
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        params = init_params
        
        for i in range(n_iterations):
            params = opt.step(cost, params)
            
        return params, cost(params)

    def get_solution(self, optimal_params: np.ndarray, steps: int) -> List[List[int]]:
        """
        Get the VRP solution from optimal parameters
        
        Args:
            optimal_params: Optimal QAOA parameters
            steps: Number of QAOA steps used
            
        Returns:
            routes: List of vehicle routes
        """
        # Run circuit with optimal parameters and measure all qubits
        @qml.qnode(self.dev)
        def measure_circuit(params, steps):
            # Initialize in superposition
            for wire in range(self.n_qubits):
                qml.Hadamard(wires=wire)
                
            # Implement QAOA steps
            for i in range(steps):
                self.cost_hamiltonian(params[i], range(self.n_qubits))
                self.mixer_hamiltonian(params[steps + i], range(self.n_qubits))
                
            return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
        
        measurements = measure_circuit(optimal_params, steps)
        
        # Convert measurements to binary
        binary_solution = [1 if m > 0 else 0 for m in measurements]
        
        # Convert binary solution to routes
        routes = []
        current_route = [0]  # Start at depot
        
        # Extract routes from binary solution
        for i in range(self.n_vehicles):
            while len(current_route) < self.n_locations:
                current = current_route[-1]
                next_idx = -1
                
                # Find next location in route
                for j in range(self.n_locations):
                    if j not in current_route:
                        idx = current*(self.n_locations-1) + (j if j < current else j-1)
                        if binary_solution[idx] == 1:
                            next_idx = j
                            break
                            
                if next_idx == -1:
                    break
                    
                current_route.append(next_idx)
                
            if len(current_route) > 1:
                routes.append(current_route)
                current_route = [0]  # Start new route at depot
                
        return routes

    def get_state_probabilities(self, optimal_params: np.ndarray, steps: int, n_shots: int = 1000) -> dict:
        """
        Get probability distribution of measured states
        
        Args:
            optimal_params: Optimal QAOA parameters
            steps: Number of QAOA steps used
            n_shots: Number of circuit measurements
            
        Returns:
            probabilities: Dictionary mapping state indices to their probabilities
        """
        @qml.qnode(self.dev)
        def measurement_circuit(params, steps):
            # Initialize in superposition
            for wire in range(self.n_qubits):
                qml.Hadamard(wires=wire)
                
            # Implement QAOA steps
            for i in range(steps):
                self.cost_hamiltonian(params[i], range(self.n_qubits))
                self.mixer_hamiltonian(params[steps + i], range(self.n_qubits))
                
            return qml.sample()
        
        # Take multiple shots and get state probabilities
        results = []
        for _ in range(n_shots):
            measurements = measurement_circuit(optimal_params, steps)
            # Convert binary measurements to decimal index
            state_idx = sum([m * 2**i for i, m in enumerate(measurements)])
            results.append(state_idx)
        
        # Calculate probabilities
        unique_states, counts = np.unique(results, return_counts=True)
        probabilities = {int(state): count/n_shots for state, count in zip(unique_states, counts)}
        
        return probabilities

    def find_optimal_states(self, probabilities: dict) -> List[int]:
        """
        Find indices of optimal states from probability distribution
        
        Args:
            probabilities: Dictionary of state probabilities
            
        Returns:
            optimal_states: List of optimal state indices
        """
        # Calculate cost for each state
        costs = {}
        for state_idx in probabilities.keys():
            # Convert decimal index to binary state
            binary_state = [int(x) for x in format(state_idx, f'0{self.n_qubits}b')]
            
            # Check if state is feasible (satisfies constraints)
            if self._is_feasible_state(binary_state):
                cost = self._calculate_cost(binary_state)
                costs[state_idx] = cost
        
        # Find states with minimum cost
        if not costs:
            return []
        
        min_cost = min(costs.values())
        optimal_states = [state for state, cost in costs.items() if abs(cost - min_cost) < 1e-6]
        
        return optimal_states

    def _is_feasible_state(self, binary_state: List[int]) -> bool:
        """
        Check if state satisfies VRP constraints
        """
        # Check each location is visited exactly once
        for i in range(1, self.n_locations):
            # Check outgoing edges
            outgoing = sum(binary_state[i*(self.n_locations-1):(i+1)*(self.n_locations-1)])
            if outgoing != 1:
                return False
            
            # Check incoming edges
            incoming = sum(binary_state[j*(self.n_locations-1) + (i if i < j else i-1)] 
                        for j in range(self.n_locations) if j != i)
            if incoming != 1:
                return False
        
        # Check vehicle count constraint (edges from/to depot)
        depot_outgoing = sum(binary_state[:self.n_locations-1])
        depot_incoming = sum(binary_state[i*(self.n_locations-1)] 
                            for i in range(1, self.n_locations))
        
        return depot_outgoing == self.n_vehicles and depot_incoming == self.n_vehicles

    def _calculate_cost(self, binary_state: List[int]) -> float:
        """
        Calculate cost of a given state
        """
        cost = 0
        for i in range(self.n_locations):
            for j in range(self.n_locations):
                if i != j:
                    idx = i*(self.n_locations-1) + (j if j < i else j-1)
                    if binary_state[idx] == 1:
                        cost += self.distance_matrix[i][j]
        return cost

    def visualize_distribution(self, probabilities: dict, optimal_states: List[int]):
        """
        Visualize probability distribution and highlight optimal states
        """
        import matplotlib.pyplot as plt
        
        # Create bar plot
        plt.figure(figsize=(12, 6))
        states = list(probabilities.keys())
        probs = list(probabilities.values())
        
        # Plot all states
        plt.bar(states, probs, color='gray', alpha=0.5)
        
        # Highlight optimal states
        optimal_probs = [probabilities[state] for state in optimal_states]
        plt.bar(optimal_states, optimal_probs, color='blue', 
                label='Optimal States')
        
        plt.xlabel('State Index')
        plt.ylabel('Probability')
        plt.title(f'Probability Distribution for VRP({self.n_locations},{self.n_vehicles})')
        plt.legend()
        
        # Add cost labels for optimal states
        for state in optimal_states:
            binary_state = [int(x) for x in format(state, f'0{self.n_qubits}b')]
            cost = self._calculate_cost(binary_state)
            plt.text(state, probabilities[state], f'Cost: {cost:.3f}', 
                    ha='center', va='bottom')
        
        plt.show()

# Example usage:
if __name__ == "__main__":
    # Example problem instance (4,2) from the paper
    n_locations = 4
    n_vehicles = 2
    distances = np.array([
        [0.0, 36.84, 5.06, 30.63],
        [36.84, 0.0, 24.55, 63.22],
        [5.06, 24.55, 0.0, 15.50],
        [30.63, 63.22, 15.50, 0.0]
    ])
    
    # Create and run VRP QAOA solver
    vrp_solver = VRPQAOA(n_locations, n_vehicles, distances)
    optimal_params, min_cost = vrp_solver.optimize(steps=12)
    
    # Get and visualize probability distribution
    probabilities = vrp_solver.get_state_probabilities(optimal_params, steps=12)
    optimal_states = vrp_solver.find_optimal_states(probabilities)
    
    print(f"Optimal state indices: {optimal_states}")
    vrp_solver.visualize_distribution(probabilities, optimal_states)
    
    # Print routes for optimal states
    for state_idx in optimal_states:
        binary_state = [int(x) for x in format(state_idx, f'0{vrp_solver.n_qubits}b')]
        routes = vrp_solver._binary_to_routes(binary_state)
        cost = vrp_solver._calculate_cost(binary_state)
        print(f"\nState {state_idx}:")
        print(f"Routes: {routes}")
        print(f"Cost: {cost}")

KeyboardInterrupt: 